In [41]:
import pandas as pd

In [46]:
df= pd.read_csv('CENTRALVALLEY_NO3N_conc_0_updated_zipcoded.csv')


all_zipcodes = df['Zipcode'].unique()

max_concs=[]
mean_concs=[]

max_concs_dom=[]
mean_concs_dom=[]

max_concs_mun=[]
mean_concs_mun=[]

wells_num=[]
wells_num_dom=[]
wells_num_mun=[]


for  zipcode in all_zipcodes:
    temp_df= df[df['Zipcode']==zipcode]
    max_concs+=[temp_df['GM_RESULT'].max()]
    mean_concs+=[temp_df['GM_RESULT'].mean()]
    wells_num+=[len(temp_df)]
    
    temp_df_dom=temp_df[temp_df['GM_WELL_CATEGORY']=='Domestic']
    max_concs_dom+=[temp_df_dom['GM_RESULT'].max()]
    mean_concs_dom+=[temp_df_dom['GM_RESULT'].mean()]
    wells_num_dom+=[len(temp_df_dom)]
    
    temp_df_mun=temp_df[temp_df['GM_WELL_CATEGORY']=='Municipal']
    max_concs_mun+=[temp_df_mun['GM_RESULT'].max()]
    mean_concs_mun+=[temp_df_mun['GM_RESULT'].mean()]
    wells_num_mun+=[len(temp_df_mun)]
    


In [43]:
df['GM_WELL_CATEGORY'].unique()

array(['Domestic', 'Municipal', 'Water Supply, Other'], dtype=object)

In [45]:
df_output=pd.DataFrame.from_dict({'Zipcode':all_zipcodes,
                                  'Mean Concentrations':mean_concs,
                                  'Max Concentrations': max_concs,
                                 'Mean Concs Domestic': mean_concs_dom,
                                 'Max Concs Domestic': max_concs_dom,
                                 'Mean Concs Municipal': mean_concs_mun,
                                 'Max Concs Municipal': max_concs_mun,
                                 'Number Wells': wells_num,
                                 'Number Wells Domestic': wells_num_dom,
                                 'Number Wells Municipal': wells_num_mun})
df_output

,Zipcode,Mean Concentrations,Max Concentrations,Mean Concs Domestic,Max Concs Domestic,Mean Concs Municipal,Max Concs Municipal,Number Wells,Number Wells Domestic,Number Wells Municipal
0,96035,2.811478,16.90,2.344000,3.47,2.716193,16.90,207,5,171
1,95674,3.956280,26.60,11.562500,26.60,1.221375,1.79,50,4,32
2,95988,4.063457,17.80,0.140000,0.14,4.324076,17.80,918,1,822
3,95220,3.261438,21.70,2.606644,17.40,3.081606,11.20,578,87,469
4,96055,2.371936,22.10,2.020500,7.50,2.429776,11.00,500,80,389
...,...,...,...,...,...,...,...,...,...,...
328,95835,0.000000,0.00,NaN,NaN,NaN,NaN,3,0,0
329,95762,3.626200,12.50,2.619000,11.00,NaN,NaN,10,8,0
330,95043,1.100000,1.10,NaN,NaN,1.100000,1.10,1,0,1
331,93461,16.446667,34.00,7.670000,7.67,NaN,NaN,3,1,0


In [48]:
df_output.to_csv('well_concentrations_max_mean.csv', index=False)

---

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Load data
df_category = pd.read_csv('CENTRALVALLEY_NO3N_conc_0_updated_zipcoded.csv')
master_data = gpd.read_file('../AGU_data/census_zipcode/central_valley_zip.shp')





(1) We need a list of Zip codes in the Central Valley with mean and max nitrate concentration. 

(1.1)I suggest creating two columns, one considering all wells and one considering only domestic wells. 

(1.2)It would also be good to count the number of domestic/all wells in the respective Zip codes.

(2) We don’t necessarily need this for the data pull of the health data, but I think it might be interesting for our analysis to also look at the number of years/months since legally mandated thresholds were first exceeded (e.g., this Zip code first exceeded 10 mg/L 5 years ago or something like that). Our health data is from 2018, so let’s only consider nitrate levels until then.

(3) We need to subset our list of Zip codes by the type of water supply. Isabel, can you take the lead on that? The priority here is to identify the zip codes and/or domestic wells that households might actually rely on for drinking water.

In [2]:
df_category

,Zipcode,GM_LATITUDE,GM_LONGITUDE,geometry,GM_WELL_ID,GM_SAMP_COLLECTION_DATE,GM_RESULT,GM_WELL_CATEGORY
0,96035,40.095745,-122.148686,POINT (-183141.19383639068 233169.59745438304),AGW080022086-HOME WELL,9/16/22,2.02,Domestic
1,96035,40.063456,-122.155983,POINT (-183844.5618455451 229597.92128949612),CA5210002_003_003,10/16/00,0.50,Municipal
2,96035,40.063456,-122.155983,POINT (-183844.5618455451 229597.92128949612),CA5210002_003_003,5/6/21,0.40,Municipal
3,96035,40.063456,-122.155983,POINT (-183844.5618455451 229597.92128949612),CA5210002_003_003,1/29/93,0.99,Municipal
4,96035,40.063456,-122.155983,POINT (-183844.5618455451 229597.92128949612),CA5210002_003_003,12/6/95,0.99,Municipal
...,...,...,...,...,...,...,...,...
233703,95974,39.490261,-121.847603,POINT (-158793.0006215956 165371.2844432355),3.92924E+14,8/21/97,7.22,"Water Supply, Other"
233704,95974,39.490261,-121.847603,POINT (-158793.0006215956 165371.2844432355),3.92924E+14,7/20/06,8.11,"Water Supply, Other"
233705,95974,39.490261,-121.847603,POINT (-158793.0006215956 165371.2844432355),3.92924E+14,7/20/06,1.83,"Water Supply, Other"
233706,95974,39.490261,-121.847603,POINT (-158793.0006215956 165371.2844432355),3.92924E+14,7/11/17,1.17,"Water Supply, Other"


In [3]:
all_zipcodes = df_category['Zipcode'].unique()
len(all_zipcodes)

333

In [13]:
domestic_wells=df_category[df_category['GM_WELL_CATEGORY']=='Domestic']
domestic_wells

all_domestic_zip=domestic_wells['Zipcode'].unique()
len(all_domestic_zip)

domestic_concentrations=[]

for zipcode in all_domestic_zip:
    temp_df = domestic_wells[domestic_wells['Zipcode'] == zipcode]
    mean_value = temp_df['GM_RESULT'].mean()
    max_value = temp_df['GM_RESULT'].max()
    domestic_concentrations.append((zipcode, 'Domestic', mean_value, max_value))

# Convert list to DataFrame
results_df = pd.DataFrame(domestic_concentrations, columns=['Zipcode', 'GM_WELL_CATEGORY', 'Mean_GM_RESULT', 'Max_GM_RESULT'])
results_df
# # Export DataFrame to CSV
# # results_df.to_csv('domestic_well_stat_concentrations.csv', index=False)

# # print("The data has been successfully exported to 'domestic_well_stat_concentrations.csv'")

,Zipcode,GM_WELL_CATEGORY,Mean_GM_RESULT,Max_GM_RESULT
0,96035,Domestic,2.344000,3.47
1,95674,Domestic,11.562500,26.60
2,95988,Domestic,0.140000,0.14
3,95220,Domestic,2.606644,17.40
4,96055,Domestic,2.020500,7.50
...,...,...,...,...
215,93653,Domestic,5.750000,5.75
216,96029,Domestic,0.040000,0.04
217,95650,Domestic,1.800000,1.90
218,95762,Domestic,2.619000,11.00


In [6]:
# municipal_wells= df_category[df_category['GM_WELL_CATEGORY']=='Municipal']
# municipal_wells

# all_municipal_zip=municipal_wells['Zipcode'].unique()
# all_municipal_zip

# municipal_concentrations=[]

# for zipcode in all_municipal_zip:
#     mun_temp_df = municipal_wells[municipal_wells['Zipcode'] == zipcode]
#     mun_mean_value = mun_temp_df['GM_RESULT'].mean()
#     mun_max_value = mun_temp_df['GM_RESULT'].max()
#     municipal_concentrations.append((zipcode, 'Municipal', mun_mean_value, mun_max_value))
    
# mun_results_df = pd.DataFrame(municipal_concentrations, columns=['Zipcode', 'GM_WELL_CATEGORY', 'Mean_GM_RESULT', 'Max_GM_RESULT'])
# mun_results_df

,Zipcode,GM_WELL_CATEGORY,Mean_GM_RESULT,Max_GM_RESULT
0,96035,Municipal,2.716193,16.90
1,95674,Municipal,1.221375,1.79
2,95988,Municipal,4.324076,17.80
3,95220,Municipal,3.081606,11.20
4,96055,Municipal,2.429776,11.00
...,...,...,...,...
323,94509,Municipal,4.134783,10.00
324,95679,Municipal,0.392308,3.60
325,93234,Municipal,2.334000,5.40
326,95391,Municipal,2.500000,2.90


In [7]:


# Filter data by well categories
domestic_wells = df_category[df_category['GM_WELL_CATEGORY'] == 'Domestic']
municipal_wells = df_category[df_category['GM_WELL_CATEGORY'] == 'Municipal']
# all_wells = pd.concat([domestic_wells, municipal_wells])

# Get unique ZIP codes from all wells
all_zip_codes = df_category['Zipcode'].unique()

concentrations = []

# Calculate statistics for each ZIP code
for zipcode in all_zip_codes:
    # Filter wells by ZIP code
    domestic_zip_wells = domestic_wells[domestic_wells['Zipcode'] == zipcode]
    municipal_zip_wells = municipal_wells[municipal_wells['Zipcode'] == zipcode]
    all_zip_wells = df_category[df_category['Zipcode'] == zipcode]
    
    # Calculate mean and max for domestic wells
    domestic_mean = domestic_zip_wells['GM_RESULT'].mean() if not domestic_zip_wells.empty else 0
    domestic_max = domestic_zip_wells['GM_RESULT'].max() if not domestic_zip_wells.empty else 0
    
    # Calculate mean and max for domestic wells
    municipal_mean = municipal_zip_wells['GM_RESULT'].mean() if not municipal_zip_wells.empty else 0
    municipal_max = municipal_zip_wells['GM_RESULT'].max() if not municipal_zip_wells.empty else 0
    
    # Calculate mean and max for all wells
    all_mean = all_zip_wells['GM_RESULT'].mean() if not all_zip_wells.empty else 0
    all_max = all_zip_wells['GM_RESULT'].max() if not all_zip_wells.empty else 0
    
    # Count wells
    num_domestic_wells = len(domestic_zip_wells)
    num_municipal_wells = len(municipal_zip_wells)
    num_all_wells = len(all_zip_wells)
    
    # Append data
    concentrations.append((zipcode, domestic_mean, domestic_max, municipal_mean, municipal_max, all_mean, all_max, num_domestic_wells, num_municipal_wells, num_all_wells))

# Convert list to DataFrame
results_df = pd.DataFrame(concentrations, columns=[
    'Zipcode', 'Domestic Mean', 'Domestic Max', 'Municipal Mean', 'Municipal Max', 'All Wells Mean', 'All Wells Max', 'Number of Domestic Wells', 'Number of Municipal Wells', 'Number of All Wells'
])

In [8]:
results_df

,Zipcode,Domestic Mean,Domestic Max,Municipal Mean,Municipal Max,All Wells Mean,All Wells Max,Number of Domestic Wells,Number of Municipal Wells,Number of All Wells
0,96035,2.344000,3.47,2.716193,16.90,2.811478,16.90,5,171,207
1,95674,11.562500,26.60,1.221375,1.79,3.956280,26.60,4,32,50
2,95988,0.140000,0.14,4.324076,17.80,4.063457,17.80,1,822,918
3,95220,2.606644,17.40,3.081606,11.20,3.261438,21.70,87,469,578
4,96055,2.020500,7.50,2.429776,11.00,2.371936,22.10,80,389,500
...,...,...,...,...,...,...,...,...,...,...
328,95835,0.000000,0.00,0.000000,0.00,0.000000,0.00,0,0,3
329,95762,2.619000,11.00,0.000000,0.00,3.626200,12.50,8,0,10
330,95043,0.000000,0.00,1.100000,1.10,1.100000,1.10,0,1,1
331,93461,7.670000,7.67,0.000000,0.00,16.446667,34.00,1,0,3


In [9]:
results_df.to_csv('well_concentrations_by_zipcode_updated.csv', index=False)